In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install -U "tensorflow>=2.5"

import os
import pathlib
import time
import csv
import json
import xml.etree.ElementTree as ET
import tensorflow as tf
from collections import namedtuple
import glob

import matplotlib
import matplotlib.pyplot as plt

import re
from shutil import copyfile
import argparse
import math
import random

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
# import tensorflow_hub as hub

%load_ext autoreload
%autoreload 2

# %matplotlib inline

tf.get_logger().setLevel('ERROR')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# tf_od API 다운로드
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 82261, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 82261 (delta 87), reused 143 (delta 68), pack-reused 82094
Receiving objects: 100% (82261/82261), 596.44 MiB | 18.04 MiB/s, done.
Resolving deltas: 100% (58722/58722), done.


In [ ]:
# coco API 설치
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
# 워크스페이스 디렉토리 생성
%%bash
# mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/scripts"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/scripts/preprocessing"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/exported-models"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/images"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/annotations"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/images/test"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/images/train"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models"
mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/pre-trained-models"


In [ ]:
!cp "/content/gdrive/MyDrive/da/scripts/preprocessing/make_tfrecord.py" \
"/content/gdrive/MyDrive/Team.B_Project/scripts/preprocessing/make_tfrecord.py"

In [ ]:
# 데이터셋 디렉토리 생성 및 해당 경로에 데이터셋 다운로드
!mkdir "/content/gdrive/MyDrive/datasets/"
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar -P "/content/gdrive/MyDrive/datasets/"

In [ ]:
# 현재 working 디렉토리에 복사 및 묶음 해제
!cp "/content/gdrive/MyDrive/datasets/VOCtrainval_11-May-2012.tar" ./
!tar -xvf ./VOCtrainval_11-May-2012.tar

In [ ]:
%cd /content

/content


In [ ]:
# 해당 경로로 이동 후 파일 갯수 확인
%cd ./VOCdevkit/VOC2012/JPEGImages/
!ls -l | grep ^- | wc -l

In [ ]:
# 이미지 디렉토리 생성 후 특정 포멧의 파일 리스트를 'images'로 바인딩
imageDir = "/content/VOCdevkit/VOC2012/JPEGImages"
images = [f for f in os.listdir(imageDir)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]        # 리눅스 정규식 표현 : 파일에 a-z,A-Z,0-9라 적혀있는 파일이면 모두 가져온다.

In [ ]:
# training set, val set 분류 비율
ratio=0.1

In [ ]:
# 세트 분류 및 확인
num_images = len(images)
num_test_images = math.ceil(ratio*num_images)
num_train_images = num_images - num_test_images

print("num_images=", num_images)
print("num_test_images=", num_test_images)
print("num_train_images=", num_train_images)

num_images= 17125
num_test_images= 1713
num_train_images= 15412


In [ ]:
# 디렉토리 지정
train_dir = "/content/train"
test_dir = "/content/test"

In [ ]:
# pascal voc format dataset only
source = imageDir
copy_xml=True
sourceXML="/content/VOCdevkit/VOC2012/Annotations"
# test set xml 파일 분류
for i in range(num_test_images):
  idx = random.randint(0, len(images)-1)
  filename = images[idx]
  copyfile(os.path.join(source, filename), os.path.join(test_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    copyfile(os.path.join(sourceXML, xml_filename), os.path.join(test_dir,xml_filename))

  images.remove(images[idx])

In [ ]:
# test set 갯수 확인 (images * 2)
%%bash
cd "/content/train"
ls -l | grep ^- | wc -l

30824


In [ ]:
# training set xml 파일 분류
for filename in images:
  copyfile(os.path.join(source, filename), os.path.join(train_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    print(copyfile(os.path.join(sourceXML, xml_filename), os.path.join(train_dir, xml_filename)))

In [ ]:
# test set 갯수 확인 (images * 2)
%%bash
cd "/content/test"
ls -l | grep ^- | wc -l

In [ ]:
# OD_API 레이블 맵 복사
!cp "/content/models/research/object_detection/data/pascal_label_map.pbtxt" \
"/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/annotations"

In [ ]:
# 레이블 맵 이동 및 이름 변경
!mv "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/annotations/pascal_label_map.pbtxt" \
"/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/annotations/label_map.pbtxt"

In [ ]:
# 순서대로 record 파일, 이미지 파일, xml 파일 경로 지정 (pascal voc는 이미지와 xml의 경로가 같음)
output_path = "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations"
image_dir = "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/images/train"
xml_dir = "/content"

In [ ]:
# train.record 파일 생성 명령어 (pascal voc는 이미지와 xml의 경로가 같음)
!python "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/scripts/preprocessing/make_tfrecord.py" \
-i "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/images/train" \
-l "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations/train.record"

In [ ]:
# test.record 파일 생성 명령어
!python "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/scripts/preprocessing/make_tfrecord.py" \
-i "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/images/test" \
-l "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations/test.record"

In [ ]:
copyfile("/content/VOCdevkit/VOC2012/JPEGImages/2007_000027.jpg", "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/images/test/2007_000027.jpg")

In [ ]:
# 모델 다운로드
%cd /content
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz

In [ ]:
!cp /content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/pre-trained-models/efficientdet_d1_coco17_tpu-32/pipeline.config -P /content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3/pipeline.config

In [ ]:
# 모델을 해당 경로에 묶음 해제
!tar xvfzp "/content/efficientdet_d1_coco17_tpu-32.tar.gz" \
-C "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/pre-trained-models"

In [ ]:
# 모델을 트레이닝할 디렉토리 생성
!mkdir "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3"
!mkdir "/content/gdrive/MyDrive/TF_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/exported-models/saved_efficientdet_d3"

In [ ]:
# train set 학습
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3" \
--pipeline_config_path="/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3/pipeline.config"

In [ ]:
# test set 학습
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3" \
--pipeline_config_path="/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3/pipeline.config" \
--checkpoint_dir="/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models"

In [ ]:
# train set 저장
!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3/pipeline.config" \
--trained_checkpoint_dir "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/models/myft_efficientdet_d3" \
--output_directory "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/exported-models"

2023-03-17 08:28:07.879082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 08:28:07.879189: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 08:28:07.879210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-17 08:28:12.142386: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environme

In [ ]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
# bbox 시각화 전처리
PATH_TO_MODEL_DIR = "/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/exported-models"
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print(PATH_TO_SAVED_MODEL)

/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/exported-models/saved_model


In [ ]:
PATH_TO_LABELS = '/content/gdrive/MyDrive/Team.B_Project/tf_efficientdet_lite/workspace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# bbox 시각화 전처리
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 29.579161882400513 seconds


In [ ]:
# 실질적인 bbox 시각화
IMAGE_PATHS_SELS=["/content/models/s01004453.jpg"]


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in IMAGE_PATHS_SELS:

    print('Running inference for {}... '.format(image_path), end='')

    # image_path = TEST_IMAGE_PATH + image_path
    image_np = load_image_into_numpy_array(image_path)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]
    input_tensor = input_tensor[..., :3]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()[..., :3]

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.10,
          agnostic_mode=False)

    plt.figure(figsize=(15,15))
    plt.imshow(image_np_with_detections)
    print('Done')

plt.show()

In [ ]:
import os
import glob
import pandas as pd
import csv

def xml_to_csv():
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob("/content/train" + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        filename = root.find('filename').text
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)
        for member in root.findall('object'):
            bndbox = member.find('bndbox')
            xminf = float(bndbox.find('xmin').text)
            yminf = float(bndbox.find('ymin').text)
            xmaxf = float(bndbox.find('xmax').text)
            ymaxf = float(bndbox.find('ymax').text)
            value = (filename,
                     width,
                     height,
                     member.find('name').text,
                     int(xminf),
                     int(yminf),
                     int(xmaxf),
                     int(ymaxf)                    
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)

    return xml_df

def main():
    for directory in ['pascal_train']:
        csv_df = xml_to_csv()
        csv_df.to_csv('/content/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted json to csv.')

main()

In [ ]:
def json_to_csv():
    path_to_json = '/content/json'
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    path_to_jpeg = '/content/jpg'
    jpeg_files = [pos_jpeg for pos_jpeg in os.listdir(path_to_jpeg) if pos_jpeg.endswith('.jpg')]
    fjpeg=(list(reversed(jpeg_files)))
    csv_list = []
    labels=[]
    for j in json_files:
        with open('/content/json/{}'.format(j), encoding='utf-8') as data_file:
            data = json.load(data_file)
            width,height=data['image']["imsize"][0],data['image']["imsize"][0]
            for item in data["annotation"]:
                box = item["box"]
                if item["class"]!='None':
                    name=item["class"]
                    # labels.append(name)
                    xmin=box[0]
                    ymin=box[1]
                    xmax=box[2]
                    ymax=box[3]
                    value = (fjpeg,
                             width,
                             height,
                             name,
                             xmin,
                             ymin,
                             xmax,
                             ymax
                             )
                    csv_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    csv_df = pd.DataFrame(csv_list, columns=column_name)
    # labels_train=list(set(labels))
    # with open("train_labels.csv", "wb") as fp:   #Pickling
    #     pickle.dump(labels_train, fp)
    # return csv_df

def main():
    for directory in ['train']:
        csv_df = json_to_csv()
        csv_df.to_csv('/content/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted json to csv.')

main()

In [ ]:
import os
import csv

folder_path = "/Users/yuyeon/Downloads/공공데이터/2D Object Detection/labels_class_5"
save_folder_path = "/Users/yuyeon/Downloads/공공데이터/2D Object Detection"
csv_file_path = os.path.join(save_folder_path, "labels_class_5.csv")
csv_data = []

for filename in os.listdir(folder_path):
    
    if filename.endswith(".txt"):
    
        txt_file_path = os.path.join(folder_path, filename)
        
        with open(txt_file_path, "r") as txt_file:
            
            for line in txt_file:
                
                data = line.strip().split()
                csv_data.append([filename[:-4] + ".jpg", data[4], data[0], data[1], data[2], data[3]])
    
with open(csv_file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filename", "class", "xmin", "ymin", "xmax", "ymax"])
    writer.writerows(csv_data)